# Virtual Try-On: Mask Shape

The `maskShape` parameter controls the shape of any auto-generated mask (created with either the `"GARMENT"` or `"PROMPT"` `maskType`). It offers `"CONTOUR"` and `"BOUNDING_BOX"` options.

When using the `"GARMENT"` mask type, Nova Canvas creates a mask that adheres closely to the body contour of the person in the source image by default. There are some cases where this default behavior is not desirable, producing an inaccurate fit, such as:

- Using a loose fitting dress as the reference image
- Adding pants to an image of someone wearing leggings or shorts
- Swapping shoes/boots of a significantly different style (e.g. high heels to flats)

For these cases, you can set the `maskShape` paremeter to `"BOUNDING_BOX"` for better results.

**Input**:

<div style="display: flex; gap: 2em; max-height: 500px; margin-right: 2em; margin-bottom: 1em">
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/vto-images/vto_mask_shape_source.jpg">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/vto-images/vto_mask_shape_reference.jpg">
    <strong><tt>referenceImage</tt></strong>
  </div>
</div>

**Output**:

<div style="display: flex; gap: 2em; max-height: 500px; margin-right: 2em; margin-bottom: 1em">
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/vto-images/vto_mask_shape_CONTOUR_output.png">
    <code>maskShape: "CONTOUR"</code> (default)
  </div>
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/vto-images/vto_mask_shape_BOUNDING_BOX_output.png">
    <code>maskShape: "BOUNDING_BOX"</code>
  </div>
</div>

_✏️ **NOTE:** When using the `"BOUNDING_BOX"` mask shape, the pose of the person in the source image may change. This is because the bounding box mask essentially "hides" the original pose details from Nova Canvas. In cases where you want to preserve the original pose <u>and</u> use a bounding box mask, you can achieve this using the `maskExclusions.preserveBodyPose` parameter discussed in a later notebook._

_⚠️ **WARNING:** In this example, you may have noticed the the `"BOUNDING_BOX"` exhibits a slight rectangular discoloration of the background, making the rectangular mask area perceptible. This is an artifact that can occur when using the bounding box shape and is more likely to occur when the background is a solid color. To avoid this, you can use a `mergeStyle` parameter of `"SEAMLESS"` which is demonstrated later in this notebook._


In [ ]:
import logging
from datetime import datetime
from random import randint
import base64
from amazon_image_gen import BedrockImageGenerator
import file_utils

logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_mask_shape_source.jpg"
reference_image_path = "../images/vto-images/vto_mask_shape_reference.jpg"

# Load the source image from disk.
with open(source_image_path, "rb") as image_file:
    source_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
# Load the reference image from disk.
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

seed = randint(0, 2147483646)

for mask_shape in ["CONTOUR", "BOUNDING_BOX"]:
    print(f'Generating image with maskShape: "{mask_shape}"...')
    inference_params = {
        "taskType": "VIRTUAL_TRY_ON",
        "virtualTryOnParams": {
            "sourceImage": source_image_base64,
            "referenceImage": reference_image_base64,
            "maskType": "GARMENT",
            "garmentBasedMask": {
                "garmentClass": "FULL_BODY",
                "maskShape": mask_shape,
            },
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "standard",
            "cfgScale": 4.5,
            "seed": seed,
        },
    }

    # Define an output directory with a unique name.
    generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_directory = f"output/{generation_id}"

    # Create the generator.
    generator = BedrockImageGenerator(output_directory=output_directory)

    # Generate the image(s).
    response = generator.generate_images(inference_params)

    if "images" in response:
        # Save and display each image
        images = file_utils.save_base64_images(
            response["images"], output_directory, f"image_{mask_shape}"
        )
        for image in images:
            display(image)